# Отбор на стажировку 2022 стажёр-data scientist AliExpress Россия, 1 этап тестирование "машинное обучение"
**Этап 1. Стажёр-data scientist**


**Вопрос 19**
Предобработка данных и их влияние на модель.

Продемонстрировать любым из алгоритмов (например, sklearn.GBM, XGBoost, LGBM) на каком-либо датасете из sklearn, например, wine, boston, diabets.

Показать код, результаты и их интерпретацию.

**Гипотеза**

1. Деревьям вёравно на масштаб данных. Но поскольку здесь для поиска параметров алгоритма применяется градиентный метод, то масштаб данных будет играть роль. Из опыта я знаю, что обычно масштабирование улучшает результаты.

2. проверку будем делать с помощью кроссвалидации на всём датасете. при чистке сделаем соответственно разделение на тест/обучающую выборку


**Заключение**
1. на голых данных - На тесте большой разброс качества
1. Сделал чистку по выбросам (выше 0.99 квантиля) - выбросили около 20 элементов
2. после чистки на тесте значительно снилизся раброс, увеличилось среднее качество
3. некоторые методы нормализации уменьшают разброс метрики по сравнению с исходными данными.


**Бейзлайн на сырых данных**

| Метод | Mean | Std |
| --- | --- | --- |
|Train	| 0.9766060454727914 | 0.0010930562077840602 |
|Test|	 0.5315287596286704 | 0.3998561954145106 |

**После чистки**

| Метод | Mean | Std |
| --- | --- | --- |
|Train	| 0.9818250630222942 | 0.0012934629706688743
|Test	| 0.8727859685014053 | 0.023656194503395227

**RobustScale сырых данных**

| Метод | Mean | Std |
| --- | --- | --- |
|Train	| 0.9766060454727914 | 0.0010930562077840602
|Test	| 0.5309324905983959 | 0.40029676204826053


**Затраты времени**

Всего потрачено времени: 3 часа

Чтобы понять что хотят в задаче около: 2 часов

Набор кода: 0.5 часа

Основное решение: 0.5 часа

Подключаем библы

In [1]:
from sklearn import ensemble
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import r2_score

from sklearn import preprocessing  # модуль с фукнциями препроцессинга

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Загружаем данные

In [2]:
from sklearn.datasets import load_boston
data = load_boston()
    
x = pd.DataFrame(data.data, columns=data['feature_names'])
y = data.target

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=5)

# print(data['DESCR'])
# x.head()

In [3]:
# x.describe()

In [4]:
# sns.pairplot(x, size=2.5)
# plt.tight_layout()

### Бейзлан без обработки данных

In [5]:
estGBR = ensemble.GradientBoostingRegressor(random_state=5)

In [6]:
scores = cross_validate(estGBR, x,y, scoring='r2', cv=13, n_jobs=-1, return_train_score=True)
print('Train\t', scores['train_score'].mean(), scores['train_score'].std())
print('Test\t', scores['test_score'].mean(), scores['test_score'].std())

# estGBR.fit(x_train, y_train)
# y_pred = estGBR.predict(x_test)
# r2_score(y_test, y_pred)

Train	 0.9766060454727914 0.0010930562077840602
Test	 0.5315287596286704 0.3998561954145106


**Заключение:**

На тесте большой разброс качества

## Чистка данных

In [7]:
# cols = ['ZN', 'B']   # колонки с наибольшим разбросом
def clean(x_train, y_train):
    cols = data.feature_names
    q = x[cols].quantile(.99).values
#     print(q)

    x_tr = x_train.copy()
    y_tr = y_train.copy()
    
# удалить дубликаты
    x_tr = x_tr.drop_duplicates()
    
#     Удалить выборсы в данных
    for i in range(len(q)):
        indx = x_tr[cols[i]] <= q[i]
        x_tr = x_tr[indx]
        y_tr = y_tr[indx]

    return x_tr, y_tr

x_tr, y_tr = clean(x_train, y_train)
x_tr.shape

(334, 13)

**Заключение**

При чистке выбросили около 20 элементов

### Прогноз после чистки

In [8]:
scores_train = []
scores_test = []

for i in [15,16,17,24,475, 234, 832, 854, 345, 27, 23]:
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=i)
    x_tr, y_tr = clean(x_train, y_train)
    
    estGBR = ensemble.GradientBoostingRegressor(random_state=5)
    estGBR.fit(x_tr, y_tr)
    
    y_pred = estGBR.predict(x_tr)
    sc = r2_score(y_tr, y_pred)
    scores_train.append(sc)
    
    y_pred = estGBR.predict(x_test)
    sc = r2_score(y_test, y_pred)
    scores_test.append(sc)
    
print('Train\t', np.mean(scores_train), np.std(scores_train))
print('Test\t', np.mean(scores_test), np.std(scores_test))
    

Train	 0.9818250630222942 0.0012934629706688743
Test	 0.8727859685014053 0.023656194503395227


**Заключение:**

На тесте разброс значений метрики меньше чем без чистки.

## Нормализация данных

Для этого примера используются исходные не очищенные данные

In [9]:
estGBR = ensemble.GradientBoostingRegressor(random_state=5)

x_tr = preprocessing.normalize(x_train, norm='l1')
x_te = preprocessing.normalize(x_test, norm='l1')

estGBR.fit(x_tr, y_train)
y_pred = estGBR.predict(x_te)
r2_score(y_test, y_pred)

0.8110471882705071

In [10]:
x_t = preprocessing.normalize(x, norm='l1')
scores = cross_validate(estGBR, x_t,y, scoring='r2', cv=13, n_jobs=-1, return_train_score=True)
print('Train\t', scores['train_score'].mean(), scores['train_score'].std())
print('Test\t', scores['test_score'].mean(), scores['test_score'].std())

Train	 0.9707210433661071 0.0015667778933249662
Test	 0.4944030996024675 0.2843994809370431


### minmax_scale

In [11]:
estGBR = ensemble.GradientBoostingRegressor(random_state=5)

x_tr = preprocessing.minmax_scale(x_train)
x_te = preprocessing.minmax_scale(x_test)

estGBR.fit(x_tr, y_train)
y_pred = estGBR.predict(x_te)
r2_score(y_test, y_pred)

0.7766376579770594

In [12]:
x_t = preprocessing.minmax_scale(x)
scores = cross_validate(estGBR, x_t,y, scoring='r2', cv=13, n_jobs=-1, return_train_score=True)
print('Train\t', scores['train_score'].mean(), scores['train_score'].std())
print('Test\t', scores['test_score'].mean(), scores['test_score'].std())

Train	 0.9766060454727914 0.0010930562077840602
Test	 0.5309324905983959 0.40029676204826053


### RobustScale

В следующие варианте важно "обучить" трансформер на выборке обучения, чтобы был один масштаб. Если этого не сделать (код в комментариях), то будут использованы данные с разными масштабами, поскольку выборки обучения и теста могут быть разные.

In [13]:
estGBR = ensemble.GradientBoostingRegressor(random_state=5)

# x_tr = preprocessing.robust_scale(x_train)
# x_te = preprocessing.robust_scale(x_test)

scl = preprocessing.RobustScaler()
scl.fit(x_train)
x_tr = scl.transform(x_train)
x_te = scl.transform(x_test)

estGBR.fit(x_tr, y_train)
y_pred = estGBR.predict(x_te)
r2_score(y_test, y_pred)

0.8292623696817077

In [14]:
x_t = preprocessing.minmax_scale(x)
scores = cross_validate(estGBR, x_t,y, scoring='r2', cv=13, n_jobs=-1, return_train_score=True)
print('Train\t', scores['train_score'].mean(), scores['train_score'].std())
print('Test\t', scores['test_score'].mean(), scores['test_score'].std())

Train	 0.9766060454727914 0.0010930562077840602
Test	 0.5309324905983959 0.40029676204826053


**Заключение:**

некоторые методы нормализации уменьшают разброс метрики по сравнению с исходными данными.